In [17]:
import pandas as pd
import numpy as np
from dotenv import dotenv_values, find_dotenv
from datacleaning.functions import pce_tables_clean, inputoutput_clean, requirements_clean
import os
# from datacleaning.functions import filter_by_granularity
from statsmodels.tsa.api import VAR
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tools.eval_measures import rmse, aic
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
from matplotlib.colors import ListedColormap
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

In [18]:
expenditures = pd.read_excel(path_rawdata + '2_4_5U.xlsx')
expenditures = expenditures.iloc[2:]
# Drop the first column
expenditures = expenditures.drop(expenditures.columns[0], axis=1)
# Remove the final 6 rows
expenditures = expenditures.iloc[:-6]

# Create new column headers
new_headers = ["Product"] 
new_headers

for col in expenditures.columns[1:]:
    new_header = str(expenditures[col][2]) + " " + expenditures[col][3]
    new_headers.append(new_header)

expenditures.columns = new_headers
expenditures = expenditures.iloc[2:].reset_index(drop=True)

def find_granularity(s):
    num_space = len(s) - len(s.lstrip())
    if num_space == 0: 
        return 1
    else: 
        return num_space/4 + 1  

granularity = [] 
granularity = expenditures['Product'].apply(find_granularity)
granularity[0] = 0

expenditures.insert(0, 'granularity', granularity)
expenditures

,granularity,Product,1959 Q1,1959 Q2,1959 Q3,1959 Q4,1960 Q1,1960 Q2,1960 Q3,1960 Q4,...,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3
0,0.0,Personal consumption expenditures,309449,315505,320725,322842,326364,332208,332126,334024,...,15950863,16285113,16718195,17030634,17415115,17684189,17917039,18269569,18419007,18679496
1,1.0,Goods,169723,172617,174524,173564,175060,178396,177503,177103,...,5536377,5515884,5732790,5879255,6014444,6046813,6047645,6133839,6144657,6231764
2,2.0,Durable goods,43649,45465,46336,44103,45455,46434,45922,44690,...,2058403,1972724,2070181,2120715,2122910,2143128,2129012,2194859,2193586,2204485
3,3.0,Motor vehicles and parts,18090,19335,20073,17658,19343,19942,20130,18912,...,745148,670962,715448,735274,725941,728151,733853,776235,772653,764572
4,4.0,New motor vehicles (55),13349,14077,14962,12103,14637,14589,15030,13610,...,383618,319872,341752,379484,364314,371760,383351,406566,398874,393760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,4.0,Market-based PCE household mainten...,---,---,---,---,---,---,---,---,...,93119,93769,95324,96735,98997,100491,100771,101895,103232,104310
405,1.0,Market-based PCE food and energy,---,---,---,---,---,---,---,---,...,1899313,1965503,2042513,2139089,2251034,2235138,2242369,2208276,2186538,2244136
406,1.0,Market-based PCE excluding food,---,---,---,---,---,---,---,---,...,12643722,12910036,13255064,13530888,13840474,14031740,14204861,14502277,14633700,14864897
407,1.0,Market-based PCE excluding energy,---,---,---,---,---,---,---,---,...,13296439,13537879,13869862,14098283,14352058,14613721,14818920,15153904,15314022,15514947


In [19]:
index_to_remove = expenditures.index[expenditures['Product'] == 'Additional aggregates:']
expenditures = expenditures.iloc[:index_to_remove[0]]

stuff_to_remove = ["Less", "to households", "Foreign travel in the United States", 'Medical expenditures of foreigners',
                    'Expenditures of foreign students in the United States']

expenditures = expenditures[~expenditures['Product'].str.contains('|'.join(stuff_to_remove), na=False)]
expenditures = expenditures.reset_index(drop = True, inplace=False)

expenditures['Product'] = expenditures['Product'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

expenditures['Product'] = expenditures['Product'].str.strip()
difference = expenditures.copy()

In [20]:
def expenditure_by_granularity(df, granularity_num):

    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    sums = df[df['granularity'] == granularity_num].iloc[:, 2:].sum(skipna=True)

    new_row_values = [-1] + [f"Expenditure by Granularity {granularity_num}"] + list(sums)

    new_row = pd.Series(new_row_values, index=df.columns)
    df = pd.concat([new_row.to_frame().T, df], ignore_index=True)
    return df

test = expenditure_by_granularity(expenditures, 5)

# expenditures
test

,granularity,Product,1959 Q1,1959 Q2,1959 Q3,1959 Q4,1960 Q1,1960 Q2,1960 Q3,1960 Q4,...,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3
0,-1,Expenditure by Granularity 5,298809.0,304678.0,309769.0,311579.0,315203.0,321330.0,321278.0,323422.0,...,16223347.0,16579491.0,17028948.0,17333750.0,17725227.0,18010068.0,18267068.0,18643996.0,18817256.0,19091468.0
1,0.0,Personal consumption expenditures,309449.0,315505.0,320725.0,322842.0,326364.0,332208.0,332126.0,334024.0,...,15950863.0,16285113.0,16718195.0,17030634.0,17415115.0,17684189.0,17917039.0,18269569.0,18419007.0,18679496.0
2,1.0,Goods,169723.0,172617.0,174524.0,173564.0,175060.0,178396.0,177503.0,177103.0,...,5536377.0,5515884.0,5732790.0,5879255.0,6014444.0,6046813.0,6047645.0,6133839.0,6144657.0,6231764.0
3,2.0,Durable goods,43649.0,45465.0,46336.0,44103.0,45455.0,46434.0,45922.0,44690.0,...,2058403.0,1972724.0,2070181.0,2120715.0,2122910.0,2143128.0,2129012.0,2194859.0,2193586.0,2204485.0
4,3.0,Motor vehicles and parts,18090.0,19335.0,20073.0,17658.0,19343.0,19942.0,20130.0,18912.0,...,745148.0,670962.0,715448.0,735274.0,725941.0,728151.0,733853.0,776235.0,772653.0,764572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,4.0,"Religious organizations, gross output",3541.0,3664.0,3761.0,3818.0,3793.0,3680.0,3572.0,3511.0,...,108147.0,110732.0,112678.0,113957.0,114119.0,115409.0,117261.0,117712.0,118182.0,119876.0
341,4.0,Foundations and grantmaking and giving establi...,75.0,79.0,81.0,85.0,104.0,106.0,118.0,124.0,...,42890.0,42038.0,45745.0,46732.0,42002.0,45808.0,47618.0,49700.0,51658.0,49569.0
342,4.0,"Social advocacy establishments, gross output",74.0,73.0,74.0,77.0,77.0,78.0,79.0,79.0,...,31707.0,30679.0,33150.0,33829.0,31578.0,33708.0,35030.0,36562.0,38002.0,36465.0
343,4.0,"Civic and social organizations, gross output",199.0,203.0,206.0,209.0,210.0,210.0,210.0,209.0,...,13368.0,12919.0,14016.0,14441.0,13289.0,14172.0,14761.0,15407.0,16014.0,15366.0


In [21]:
residuals = pd.read_pickle(path_cleandata + 'inversions//residuals.pkl')
products = pd.DataFrame(residuals["product"].unique())
products.columns = ['Product']
products

,Product
0,Accessories and parts
1,Air transportation
2,Alcohol in purchased meals
3,"Amusement parks, campgrounds, and related recr..."
4,"Audio discs, tapes, vinyl, and permanent digit..."
...,...
142,Water supply and sewage maintenance
143,Water transportation
144,Window coverings
145,Wine


In [22]:
total_expenditure = pd.DataFrame([{"Product": "Personal consumption expenditures"}])
products = pd.concat([total_expenditure, products], ignore_index=True)
included_products = expenditures.merge(products, on="Product", how = "inner" )
included_products

,granularity,Product,1959 Q1,1959 Q2,1959 Q3,1959 Q4,1960 Q1,1960 Q2,1960 Q3,1960 Q4,...,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3
0,0.0,Personal consumption expenditures,309449.0,315505.0,320725.0,322842.0,326364.0,332208.0,332126.0,334024.0,...,15950863.0,16285113.0,16718195.0,17030634.0,17415115.0,17684189.0,17917039.0,18269569.0,18419007.0,18679496.0
1,6.0,New domestic autos,11794.0,12393.0,13271.0,10466.0,13040.0,13122.0,13647.0,12383.0,...,40735.0,35383.0,36670.0,38021.0,35893.0,37483.0,37765.0,35355.0,37119.0,35058.0
2,6.0,New foreign autos,1114.0,1208.0,1223.0,1244.0,1139.0,996.0,925.0,816.0,...,17452.0,15916.0,11946.0,17167.0,17734.0,17649.0,18373.0,19276.0,19281.0,18881.0
3,5.0,New light trucks,441.0,476.0,468.0,393.0,458.0,471.0,458.0,411.0,...,325431.0,268572.0,293136.0,324295.0,310688.0,316628.0,327212.0,351935.0,342474.0,339822.0
4,5.0,Tires,1481.0,1504.0,1549.0,1538.0,1556.0,1590.0,1560.0,1566.0,...,43403.0,43009.0,44469.0,45870.0,47525.0,48243.0,48977.0,49093.0,50511.0,51333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,5.0,"Moving, storage, and freight services",305.0,323.0,341.0,355.0,358.0,353.0,345.0,339.0,...,21120.0,21494.0,22200.0,22662.0,23021.0,22738.0,21916.0,21725.0,20937.0,20725.0
144,5.0,"Repair of furniture, furnishings, and floor co...",372.0,383.0,391.0,398.0,398.0,390.0,379.0,369.0,...,1480.0,1369.0,1333.0,1275.0,1353.0,1361.0,1459.0,1515.0,1583.0,1674.0
145,5.0,Repair of household appliances,201.0,281.0,240.0,197.0,266.0,267.0,265.0,264.0,...,9007.0,8938.0,8863.0,8492.0,8856.0,8971.0,9318.0,9698.0,10215.0,10602.0
146,5.0,Other household services,58.0,61.0,64.0,66.0,67.0,66.0,65.0,66.0,...,31835.0,31686.0,31809.0,32147.0,32643.0,33601.0,33739.0,34251.0,35520.0,36106.0


In [23]:
sum_expend = included_products.iloc[1:,2:].sum()
new_calc = [-1] + ["Expenditure Chosen"] + list(sum_expend)
new_calc = pd.Series(new_calc, index=included_products.columns)
new_calc
included_products = pd.concat([new_calc.to_frame().T, included_products], ignore_index=True)
included_products

,granularity,Product,1959 Q1,1959 Q2,1959 Q3,1959 Q4,1960 Q1,1960 Q2,1960 Q3,1960 Q4,...,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3
0,-1,Expenditure Chosen,208120.0,211907.0,215242.0,214809.0,217935.0,221549.0,220876.0,221153.0,...,7802283.0,7985551.0,8264348.0,8509579.0,8765231.0,8874971.0,8987171.0,9114571.0,9137980.0,9272621.0
1,0.0,Personal consumption expenditures,309449.0,315505.0,320725.0,322842.0,326364.0,332208.0,332126.0,334024.0,...,15950863.0,16285113.0,16718195.0,17030634.0,17415115.0,17684189.0,17917039.0,18269569.0,18419007.0,18679496.0
2,6.0,New domestic autos,11794.0,12393.0,13271.0,10466.0,13040.0,13122.0,13647.0,12383.0,...,40735.0,35383.0,36670.0,38021.0,35893.0,37483.0,37765.0,35355.0,37119.0,35058.0
3,6.0,New foreign autos,1114.0,1208.0,1223.0,1244.0,1139.0,996.0,925.0,816.0,...,17452.0,15916.0,11946.0,17167.0,17734.0,17649.0,18373.0,19276.0,19281.0,18881.0
4,5.0,New light trucks,441.0,476.0,468.0,393.0,458.0,471.0,458.0,411.0,...,325431.0,268572.0,293136.0,324295.0,310688.0,316628.0,327212.0,351935.0,342474.0,339822.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,5.0,"Moving, storage, and freight services",305.0,323.0,341.0,355.0,358.0,353.0,345.0,339.0,...,21120.0,21494.0,22200.0,22662.0,23021.0,22738.0,21916.0,21725.0,20937.0,20725.0
145,5.0,"Repair of furniture, furnishings, and floor co...",372.0,383.0,391.0,398.0,398.0,390.0,379.0,369.0,...,1480.0,1369.0,1333.0,1275.0,1353.0,1361.0,1459.0,1515.0,1583.0,1674.0
146,5.0,Repair of household appliances,201.0,281.0,240.0,197.0,266.0,267.0,265.0,264.0,...,9007.0,8938.0,8863.0,8492.0,8856.0,8971.0,9318.0,9698.0,10215.0,10602.0
147,5.0,Other household services,58.0,61.0,64.0,66.0,67.0,66.0,65.0,66.0,...,31835.0,31686.0,31809.0,32147.0,32643.0,33601.0,33739.0,34251.0,35520.0,36106.0


In [24]:
# def add_commas(x):
#     if isinstance(x, (int, float)):
#         return f"{x:,}"
#     return x

# included_products
# dfdsfd = included_products.applymap(add_commas)
# included_products

In [25]:
included_products
def add_commas(x):
    if isinstance(x, (int, float)):
        return f"{x:,}"
    return x

included_products
dfdsfd = included_products.applymap(add_commas)
dfdsfd

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_676\1805530555.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfdsfd = included_products.applymap(add_commas)


,granularity,Product,1959 Q1,1959 Q2,1959 Q3,1959 Q4,1960 Q1,1960 Q2,1960 Q3,1960 Q4,...,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3
0,-1,Expenditure Chosen,"208,120.0","211,907.0","215,242.0","214,809.0","217,935.0","221,549.0","220,876.0","221,153.0",...,"7,802,283.0","7,985,551.0","8,264,348.0","8,509,579.0","8,765,231.0","8,874,971.0","8,987,171.0","9,114,571.0","9,137,980.0","9,272,621.0"
1,0.0,Personal consumption expenditures,"309,449.0","315,505.0","320,725.0","322,842.0","326,364.0","332,208.0","332,126.0","334,024.0",...,"15,950,863.0","16,285,113.0","16,718,195.0","17,030,634.0","17,415,115.0","17,684,189.0","17,917,039.0","18,269,569.0","18,419,007.0","18,679,496.0"
2,6.0,New domestic autos,"11,794.0","12,393.0","13,271.0","10,466.0","13,040.0","13,122.0","13,647.0","12,383.0",...,"40,735.0","35,383.0","36,670.0","38,021.0","35,893.0","37,483.0","37,765.0","35,355.0","37,119.0","35,058.0"
3,6.0,New foreign autos,"1,114.0","1,208.0","1,223.0","1,244.0","1,139.0",996.0,925.0,816.0,...,"17,452.0","15,916.0","11,946.0","17,167.0","17,734.0","17,649.0","18,373.0","19,276.0","19,281.0","18,881.0"
4,5.0,New light trucks,441.0,476.0,468.0,393.0,458.0,471.0,458.0,411.0,...,"325,431.0","268,572.0","293,136.0","324,295.0","310,688.0","316,628.0","327,212.0","351,935.0","342,474.0","339,822.0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,5.0,"Moving, storage, and freight services",305.0,323.0,341.0,355.0,358.0,353.0,345.0,339.0,...,"21,120.0","21,494.0","22,200.0","22,662.0","23,021.0","22,738.0","21,916.0","21,725.0","20,937.0","20,725.0"
145,5.0,"Repair of furniture, furnishings, and floor co...",372.0,383.0,391.0,398.0,398.0,390.0,379.0,369.0,...,"1,480.0","1,369.0","1,333.0","1,275.0","1,353.0","1,361.0","1,459.0","1,515.0","1,583.0","1,674.0"
146,5.0,Repair of household appliances,201.0,281.0,240.0,197.0,266.0,267.0,265.0,264.0,...,"9,007.0","8,938.0","8,863.0","8,492.0","8,856.0","8,971.0","9,318.0","9,698.0","10,215.0","10,602.0"
147,5.0,Other household services,58.0,61.0,64.0,66.0,67.0,66.0,65.0,66.0,...,"31,835.0","31,686.0","31,809.0","32,147.0","32,643.0","33,601.0","33,739.0","34,251.0","35,520.0","36,106.0"


In [26]:
check_granularity = expenditures
check_granularity.iloc[:, 2:] = 0

check_two = pd.merge(check_granularity, included_products, on="Product", how ='left', suffixes=('_left', '_right'))
columns_to_keep = ['granularity_left', 'Product'] + [col for col in check_two.columns if col.endswith('_right')]
columns_to_keep = [i for i in columns_to_keep if i != "granularity_right"]

df_result = check_two[columns_to_keep]

df_result.columns = ['granularity', 'Product'] + [col.replace('_right', '') for col in df_result.columns if col != 'Product' and col != 'granularity_left']


df_result = df_result.fillna(0)
first_quarter = df_result.iloc[:,:3]
first_quarter
# first_quarter.to_excel("cehck.xlsx")

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_676\3365008841.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_result = df_result.fillna(0)


,granularity,Product,1959 Q1
0,0.0,Personal consumption expenditures,309449.0
1,1.0,Goods,0.0
2,2.0,Durable goods,0.0
3,3.0,Motor vehicles and parts,0.0
4,4.0,New motor vehicles,0.0
...,...,...,...
339,4.0,"Religious organizations, gross output",0.0
340,4.0,Foundations and grantmaking and giving establi...,0.0
341,4.0,"Social advocacy establishments, gross output",0.0
342,4.0,"Civic and social organizations, gross output",0.0


In [27]:
first_quarter = pd.read_excel('B:\OneDrive - DAZ\Desktop\Shocks Production Networks\shocks_productionnetworks\cehck.xlsx')

def sum_values(df):
    df = df.copy()

    for i in range(len(df)-1, 0, -1):
        current_granularity = df.loc[i, 'granularity']
        sum_value = 0
        for j in range(i+1, len(df)):
            if df.loc[j, 'granularity'] == current_granularity + 1:
                sum_value += df.loc[j, '1959 Q1']
            if df.loc[j, 'granularity'] == current_granularity:
                break
        df.loc[i, '1959 Q1'] += sum_value
    

    
    first_row_sum = df[df['granularity'] == 1]['1959 Q1'].sum()
    df.at[0, '1959 Q1'] = first_row_sum


    return df


sum_filtered = sum_values(first_quarter)

# processed_df.to_excel("post6.xlsx")

sum_filtered = sum_filtered.iloc[:,2:]
sum_filtered

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\TonyGui\AppData\Local\Temp\ipykernel_676\1244413954.py:1: SyntaxWarning: invalid escape sequence '\O'
  first_quarter = pd.read_excel('B:\OneDrive - DAZ\Desktop\Shocks Production Networks\shocks_productionnetworks\cehck.xlsx')


,Product,1959 Q1
0,Personal consumption expenditures,208120
1,Goods,152905
2,Durable goods,41222
3,Motor vehicles and parts,15664
4,New motor vehicles,13349
...,...,...
339,"Religious organizations, gross output",0
340,Foundations and grantmaking and giving establi...,0
341,"Social advocacy establishments, gross output",0
342,"Civic and social organizations, gross output",0


In [28]:
products.head(30
              )

,Product
0,Personal consumption expenditures
1,Accessories and parts
2,Air transportation
3,Alcohol in purchased meals
4,"Amusement parks, campgrounds, and related recr..."
5,"Audio discs, tapes, vinyl, and permanent digit..."
6,Audio equipment
7,Bakery products
8,Beef and veal
9,Beer


In [29]:
sum_no_filter = difference.iloc[:,0:3]
filted_merge = sum_filtered.merge(sum_no_filter, on="Product", how="inner", suffixes=["_filtered", "_original"])
filted_merge = filted_merge[["granularity", "Product", "1959 Q1_original", "1959 Q1_filtered"]]

filted_merge.replace('---', 0, inplace=True)
filted_merge["difference"] = filted_merge["1959 Q1_original"] - filted_merge["1959 Q1_filtered"]




def highlight_rows(row):
    if row['Product'] in products['Product'].values:
        return ['background-color: yellow'] * len(row)
    else:
        return [''] * len(row)

filted_merge = filted_merge.style.apply(highlight_rows, axis=1)

filted_merge.to_excel("differences.xlsx")
filted_merge

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_676\1955324536.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filted_merge.replace('---', 0, inplace=True)


PermissionError: [Errno 13] Permission denied: 'differences.xlsx'